# DATA PROJECT

**Table of contents**<a id='toc0_'></a>    
- 1. [Read and clean data](#toc1_)  
    - 1.1 [Data set 1](#toc1_1_)
    - 1.2 [Data set 2](#toc1_2_)
- 2. [Exploring the data](#toc2_) 
    - 2.1 [Summary statistics of the two data sets](#toc2_1_)
    - 2.2 [Visualization](#toc2_2_)
    - 2.3 [Merging the data sets](#toc2_3_)
- 3. [Conclusion](#toc3_)

#### Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


## 1. <a id='toc1_'></a>[Read and clean data](#toc0_)

## 1.1 <a id='toc1_1_'></a>[Data set 1](#toc0_) 

In [ ]:
# Load and print data set 1
pricedata = pd.read_excel('data.xlsx', sheet_name='Gns. Pris pr ejendom', skiprows=2)
print(pricedata)

In [ ]:
# We clean up data set 1 a bit
pricedata = pd.read_excel('data.xlsx', sheet_name='Gns. Pris pr ejendom', skiprows=2)
pricedata.head(17)

In [ ]:
# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2'
pricedata_cleaned = pricedata.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'])
pricedata_cleaned.head(17)

In [ ]:
# We remove the rows which are not actual data
pricedata_cleaned = pricedata_cleaned.dropna(subset=['Unnamed: 3']).reset_index(drop=True)

# We rename the columns to have meaningful names
pricedata_cleaned = pricedata_cleaned.rename(columns={'Unnamed: 3': 'Area'})

pricedata_cleaned.head(17)

#### We want our data to be in years instead of quarters. Therefore, we will make some further changes / cleaning of our data:

In [ ]:
# Now we will aggregate the quarters into years.
# First we need to melt the dataframe to go from a wide to long format.
pricedata_melted = pd.melt(pricedata_cleaned, id_vars=['Area'], var_name='YearQuarter', value_name='Price')

pricedata_melted.head(10) # We display the first 10 rows to verify the results

In [ ]:
# Now we extract year from 'YearQuarter' and convert 'Price' to numeric
pricedata_melted['Year'] = pricedata_melted['YearQuarter'].str.extract('(\d+)')
pricedata_melted['Price'] = pd.to_numeric(pricedata_melted['Price'])

pricedata_melted.head(50) # We display the first 50 rows to verify the results

In [ ]:
# Lastly, we drop the 'YearQuarter' column as it's no longer needed
pricedata_melted.drop('YearQuarter', axis=1, inplace=True)

# We group by 'Area' and 'Year' and sum the 'Price' for each group
yearly_pricedata = pricedata_melted.groupby(['Area', 'Year'], as_index=False).sum()

yearly_pricedata.head(10)  # We display the first 10 rows to verify the results

#### We have successfully cleaned our initial dataset. We removed irrelevant information, converted our data from quarterly to yearly, and transformed it from wide to long.

## 1.2 <a id='toc1_2_'></a>[Data set 2](#toc0_)

In [ ]:
# We do the same for data set 2
# We load the data
salesdata = pd.read_excel('data.xlsx', sheet_name='Antal salg', skiprows=2)

# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2'
salesdata_cleaned = salesdata.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'])

# We remove the rows which are not actual data
salesdata_cleaned = salesdata_cleaned.dropna(subset=['Unnamed: 3']).reset_index(drop=True)

# We rename the columns to have meaningful names
salesdata_cleaned = salesdata_cleaned.rename(columns={'Unnamed: 3': 'Area'})

# Now we will aggregate the quarters into years.
# First we need to melt the dataframe to go from a wide to long format.
salesdata_melted = pd.melt(salesdata_cleaned, id_vars=['Area'], var_name='YearQuarter', value_name='Sales')

# Now we extract year from 'YearQuarter' and convert 'Price' to numeric
salesdata_melted['Year'] = salesdata_melted['YearQuarter'].str.extract('(\d+)')
salesdata_melted['Sales'] = pd.to_numeric(salesdata_melted['Sales'])

# Lastly, we drop the 'YearQuarter' column as it's no longer needed
salesdata_melted.drop('YearQuarter', axis=1, inplace=True)

# We group by 'Area' and 'Year' and sum the 'Sales' for each group
yearly_salesdata = salesdata_melted.groupby(['Area', 'Year'], as_index=False).sum()

yearly_salesdata.head(10)  # We display the first 10 rows to verify the results

#### To make our analysis more managable, we want to focus on the capital region (Region Hovedstaden) and the North Jutland region (Region Nordjylland):

In [ ]:
# We define the two areas
areas_of_interest = ['Region Hovedstaden', 'Region Nordjylland']

# We filter the DataFrame for the areas of interest
focused_pricedata = yearly_pricedata[yearly_pricedata['Area'].isin(areas_of_interest)]

# We display the some rows to verify the results
print(focused_pricedata.head(50))

# We do the same for the second data set
focused_salesdata = yearly_salesdata[yearly_salesdata['Area'].isin(areas_of_interest)]

## 2 <a id='toc2_'></a>[Exploring the data sets](#toc0_)

## 2.1 <a id='toc2_1_'></a>[Summary statistics of the two data sets](#toc0_)

#### Before setting up plots etc., we examine some basic statistics of the two data sets:

In [ ]:
# Data set 1
yearly_pricedata.describe()

In [ ]:
# Data set 2
yearly_salesdata.describe()

In [ ]:
focused_pricedata.describe()

In [ ]:
focused_salesdata.describe()

## 2.2 <a id='toc2_2_'></a>[Visualization](#toc0_)

#### For fun, we visualize the entire data set 1:

In [ ]:
# We set up the figure and axis
plt.figure(figsize=(14, 8))

# We loop through each unique area in the DataFrame and plot it
for area in yearly_pricedata['Area'].unique():
    area_data = yearly_pricedata[yearly_pricedata['Area'] == area]
    plt.plot(area_data['Year'], area_data['Price'], marker='o', label=area)

# We add title and labels
plt.title('Annual Condominium Prices Across All Regions')
plt.xlabel('Year')
plt.ylabel('Average Condominium Price (1000 Krones)')

# We add a legend to the plot
plt.legend(title='Area', bbox_to_anchor=(1.05, 1), loc='upper left')

# We rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# We add grid lines
plt.grid(True)

# We adjust layout to make room for the legend
plt.tight_layout()

# We show the plot
plt.show()

#### From the graph above we see that the condominium prices have shown a substantial increase across all regions from 1991 to 2023.

#### We move on to the visualization of the focused areas of the two data sets:

In [ ]:
# We try to visualize the data for data set 1
# We'll create a line plot for each area
for area in areas_of_interest:
    # We filter the data for the current area
    area_data = focused_pricedata[focused_pricedata['Area'] == area]
    
    # We plot the data
    plt.figure(figsize=(12, 6))  # Set the figure size
    plt.plot(area_data['Year'], area_data['Price'], marker='o', label=area)  # Plot the data
    
    # We add title and labels
    plt.title(f'Annual Condominium Prices in {area}')
    plt.xlabel('Year')
    plt.ylabel('Average Condominium Price (in 1000 DKK)')
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
    plt.legend()  # Show the legend
    plt.grid(True)  # Show grid lines for better readability
    plt.tight_layout()  # Adjust the layout

    # We show the plot
    plt.show()

#### In both Region Hovedstaden and Region Nordjylland, condominium prices have trended upwards from 1992 to 2023, with significant growth particularly from the early 2000s onwards. Region Hovedstaden shows a more volatile price trajectory with a peak around 2006 followed by a dip, and a notable rise post-2012, suggesting a more dynamic market. In contrast, Region Nordjylland's prices follow a steadier upward trajectory with less volatility, which could imply a more stable market.

In [ ]:
# We do the same for data set 2
# We'll create a line plot for each area
for area in areas_of_interest:
    # We filter the data for the current area
    area_data = focused_salesdata[focused_salesdata['Area'] == area]
    
    # We plot the data
    plt.figure(figsize=(12, 6))  # Set the figure size
    plt.plot(area_data['Year'], area_data['Sales'], marker='o', label=area)  # Plot the data
    
    # We add title and labels
    plt.title(f'Annual Number of Condominium Sales in {area}')
    plt.xlabel('Year')
    plt.ylabel('Total Number of Condominium sales')
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
    plt.legend()  # Show the legend
    plt.grid(True)  # Show grid lines for better readability
    plt.tight_layout()  # Adjust the layout

    # We show the plot
    plt.show()

#### The number of condominium sales in both Region Hovedstaden and Region Nordjylland displays significant fluctuations over the years. Region Hovedstaden shows more pronounced volatility whereas sales numbers also vary but with a less extreme range in Region Nordjylland.

#### It is difficult to compare the two regions due to the axes. Therefore, we combine the two plots for each data set:

In [ ]:
# We combine the plots for both areas in one figure for comparison (data set 1)
fig = px.line(focused_pricedata, x='Year', y='Price', color='Area',
              title='Annual Condominium Prices',
              labels={'Price': 'Average Condominium Price (1000 Krones)'})
fig.update_xaxes(tickangle=45)
fig.show()

#### Condominium prices in Region Hovedstaden have grown substantially and remain higher than in Region Nordjylland, which shows a more steady increase.

In [ ]:
# We combine the plots for both areas in one figure for comparison (data set 2)
fig = px.line(focused_salesdata, x='Year', y='Sales', color='Area',
              title='Annual Condominium Sales',
              labels={'Sales': 'Total Condominium Sales'})
fig.update_xaxes(tickangle=45)
fig.show()

#### Condominium sales in Region Hovedstaden have been considerably higher and more volatile compared to the consistently lower and steadier sales in Region Nordjylland over the years.

## 2.3 <a id='toc2_3_'></a>[Merging the data sets](#toc0_)

#### We want to merge the two data sets:

In [ ]:
# Merge the two datasets on 'Area' and 'Year' columns
merged_data = pd.merge(focused_pricedata, focused_salesdata, on=['Area', 'Year'], how='inner')

# We plot our merged data
for area in areas_of_interest:
    # We filter the data for the current area
    area_data = merged_data[merged_data['Area'] == area]
    
    # We create a new figure and a twin Y-axis for the second variable
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # We plot the 'Price' on the primary y-axis
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Average Condominium Price (1000 Krones)', color='tab:blue')
    ax1.plot(area_data['Year'], area_data['Price'], color='tab:blue', marker='o', label='Price')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    # We create a second y-axis for the 'Sales' data
    ax2 = ax1.twinx()
    ax2.set_ylabel('Number of Sales', color='tab:red')
    ax2.plot(area_data['Year'], area_data['Sales'], color='tab:red', marker='o', label='Sales')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    # Wotate the x-axis labels for better readability
    ax1.set_xticklabels(area_data['Year'], rotation=45)

    # We add a title and a grid
    plt.title(f'Annual Condominium Prices and Number of Sales in {area}')
    ax1.grid(True)

    # We show the plot with a tight layout
    fig.tight_layout()
    plt.show()

#### The visualization of the merged data is not optimal. However, it shows that the data sets can be merged.

#### We want to check the correlation between the condominium prices and number of sales:

In [ ]:
# We calculate the correlation matrix for the merged data
correlation_matrix = merged_data[['Price', 'Sales']].corr()

# We display the correlation matrix
print(correlation_matrix)

#### We want to visualize the correlation in a heatmap:

In [ ]:
# We set up the matplotlib figure
plt.figure(figsize=(8, 6))

# We draw the heatmap
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={'label': 'Correlation Coefficient'})

# We add a title
plt.title('Correlation between Condominium Prices and Number of Sales')

# We show the plot
plt.show()

#### The correlation coefficient of approx. 0.46 between condominium prices and sales numbers suggests a moderate positive relationship; as prices increase, sales numbers tend to increase as well, but the connection is not very strong.

## 3 <a id='toc3_'></a>[Conclusion](#toc0_)

#### The project successfully demonstrates data cleaning and structuring techniques, data analysis methods, and data visualization to gain insights into the Danish condominium market from 1992 to 2023. 

#### Utilizing data from statistikbanken.dk, it analyzes trends in average condominium prices and sales volumes. The moderate positive correlation of 0.46 indicates that while higher prices are somewhat associated with increased sales, the relationship is not very strong, suggesting other factors also play significant roles in sales volume. The visualizations and analyses provide an overview of the market dynamics over the specified period. 